In [25]:
import torch 
import pandas as pd 
import numpy as np 
import os 
import sys
sys.path.append('/home/jessica/labelGAN/datasetGAN_release/datasetGAN')
from label_model import latent_classifier
from train_dataset import labelDataLatent
from tqdm import tqdm
import torch.nn as nn 
device = "cpu"
from torch.utils.data import DataLoader

In [26]:
test_latent_path = '/data3/jessica/data/labelGAN/test_images/latents/'
checkpoint_path_label = '/data3/jessica/data/labelGAN/results_dir_multitask_generation_classif/'

In [27]:
p = '/data3/jessica/data/labelGAN/vinbig/train.csv'
df = pd.read_csv(p)

In [28]:
files = os.listdir(test_latent_path)
files = sorted(files)
label_data = labelDataLatent(files, test_latent_path, device, ret_id=True)
train_loader_classif = DataLoader(dataset=label_data, batch_size=10, shuffle=True)

/data3/jessica/data/labelGAN/test_images/latents/


In [29]:
ids = [elem.replace('latent_' ,"").replace(".npy", "") for elem in files]
df = df[df.image_id.isin(ids)]
df['class_name'] = df['class_name'].apply(lambda x: x.replace(" ", "").replace("/", ""))
fil = df.groupby(by='image_id').class_name.apply(list)
fil = fil.apply(lambda x: np.unique(np.array([few_shot_classes[xel] for xel in x])))

In [33]:

label_classifier_instance = latent_classifier(15).to(device)
checkpoint = torch.load(os.path.join(checkpoint_path_label, 'model_label_classif_number_' + '.pth'))
label_classifier_instance.load_state_dict(checkpoint['model_state_dict'])
label_classifier_instance.eval()

latent_classifier(
  (lin): Linear(in_features=9216, out_features=128, bias=True)
  (label_layers): Sequential(
    (0): ReLU()
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=128, out_features=32, bias=True)
    (3): ReLU()
    (4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Linear(in_features=32, out_features=15, bias=True)
  )
)

In [34]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim=1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim=1)

    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)

    acc = acc * 100

    return acc

In [35]:
few_shot_classes = {"Nofinding": 0,
    'NoduleMass': 9,
    'Infiltration': 7,
    'LungOpacity': 3,
    'Consolidation': 4,
    'Pleuralthickening': 5,
    'ILD': 6,
    'Cardiomegaly': 2,
    'Pulmonaryfibrosis': 8,
    'Aorticenlargement': 1,
    'Otherlesion': 10,
    'Pleuraleffusion': 11,
    'Calcification': 12,
    'Atelectasis': 13,
    'Pneumothorax': 14}

In [50]:
all_preds = []
all_labels = []
accs = []
losses = []
ids = []
all_probs = []
sm = nn.Softmax(dim=1)
corr_preds = []
for X_batch, _, ids in tqdm(train_loader_classif):
    labels = [fil[id] for id in ids]
    X_batch = X_batch.detach()
    y_pred = label_classifier_instance(X_batch.squeeze())
    class_pred = sm(y_pred).argmax(-1).cpu().detach()
    for i, pred in enumerate(class_pred):
        corr_preds.append(pred.item() inlabels[i])
    del X_batch, y_pred
print("TEST ACC": np.array(corr_preds).mean())

100%|██████████| 9/9 [00:00<00:00, 15.03it/s]


In [51]:
np.array(corr_preds).mean()

/tmp/ipykernel_1827645/3010653121.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(corr_preds)


array([[9, array([1, 2, 3, 7])],
       [1, array([ 1,  3,  4,  5,  7,  8, 10, 13])],
       [1, array([ 1,  2, 12])],
       [8, array([ 3, 11])],
       [1, array([ 5,  6,  7,  8, 10, 11, 13])],
       [8, array([1, 3, 4, 5, 8, 9])],
       [1, array([1, 2, 3, 5, 6, 8, 9])],
       [0, array([0])],
       [8, array([11, 14])],
       [8, array([1, 2, 8])],
       [1, array([0])],
       [8, array([ 1,  5,  6, 10, 11])],
       [1, array([ 4,  5,  7,  8,  9, 11, 13])],
       [9, array([1, 2, 5, 8])],
       [1, array([ 5,  7,  8, 13])],
       [6, array([1, 2])],
       [0, array([ 1,  2,  5, 10, 11])],
       [0, array([ 2,  3,  5,  6,  8, 10, 11])],
       [0, array([1, 5, 8, 9])],
       [1, array([ 1,  2,  3,  7,  8, 10])],
       [2, array([ 2,  3,  4,  7,  9, 11])],
       [11, array([ 2,  3,  4,  5,  7, 11])],
       [8, array([ 1,  3,  9, 10])],
       [5, array([ 3,  7,  8, 11, 13])],
       [1, array([ 1,  3,  5,  6,  7,  8, 13])],
       [5, array([ 1,  5,  8,  9, 11, 14])

In [15]:
a,b,c,d = next(iter(train_loader_classif))

In [32]:
(np.array(all_preds) == np.array(all_labels)).mean()

0.25

In [22]:
b.shape, a.shape

(torch.Size([10, 1, 18, 512]), torch.Size([10, 1, 18, 512]))

In [27]:
all_probs

[tensor(0.9854),
 tensor(1.0000),
 tensor(0.9995),
 tensor(0.9987),
 tensor(0.8178),
 tensor(0.9634),
 tensor(1.0000),
 tensor(0.9164),
 tensor(0.8237),
 tensor(0.9313),
 tensor(0.5444),
 tensor(0.9402),
 tensor(0.9990),
 tensor(0.7623),
 tensor(0.9866),
 tensor(0.7490),
 tensor(0.8728),
 tensor(0.9484),
 tensor(0.7625),
 tensor(0.9563),
 tensor(1.0000),
 tensor(1.),
 tensor(0.9842),
 tensor(0.9971),
 tensor(0.9899),
 tensor(0.9872),
 tensor(0.5705),
 tensor(1.0000),
 tensor(0.9148),
 tensor(0.9998),
 tensor(0.6106),
 tensor(0.6352),
 tensor(0.9742),
 tensor(0.4947),
 tensor(0.9953),
 tensor(0.9933),
 tensor(0.8699),
 tensor(0.9891),
 tensor(0.9264),
 tensor(0.9269),
 tensor(0.8208),
 tensor(0.9795),
 tensor(0.4131),
 tensor(0.9999),
 tensor(0.9026),
 tensor(0.9986),
 tensor(0.9229),
 tensor(0.9991),
 tensor(0.9731),
 tensor(0.7401),
 tensor(0.6989),
 tensor(0.9980),
 tensor(0.9998),
 tensor(0.9999),
 tensor(1.0000),
 tensor(0.9788),
 tensor(0.9853),
 tensor(0.9923),
 tensor(0.3498),
 